In this tutorial, we build a PDF QA bot with open source models. We show how to use commercially available opensource models to query your knowledge base.

In [1]:
### Installations
!pip install langchain==0.0.189
!pip install chromadb==0.3.25
!pip install pdfplumber==0.9.0
!pip install tiktoken==0.4.0
!pip install lxml==4.9.2
!pip install torch==2.0.1
!pip install transformers==4.29.2
!pip install accelerate==0.19.0
!pip install sentence-transformers==2.2.2
!pip install einops==0.6.1
!pip install xformers==0.0.20

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


...

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.20 requires torch==2.0.1, but you have torch 2.5.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.0.1-cp311-cp311-win_amd64.whl.metadata (24 kB)
Using cached torch-2.0.1-cp311-cp311-win_amd64.whl (172.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.20.1 requires torch==2.5.1, but you have torch 2.0.1 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import json
import os
try:
    from hyperplane.utils import is_jhub
    if is_jhub(): ##If using jupyter hub and conda base environment
        openaiKeyFile = '/root/.secret/openai_key.json'
    else:
        ## Storing as a secret on k8s cluster
        openaiKeyFile = '/etc/hyperplane/secrets/openai_key.json'
    with open(openaiKeyFile) as f:
        os.environ["OPENAI_API_KEY"] = json.load(f)['openai_key']
except Exception as e:
    print(e)
    load_dotenv()

No module named 'hyperplane'


In [3]:
EMB_SBERT_MPNET_BASE = "sentence-transformers/all-mpnet-base-v2"

In [4]:
LLM_FLAN_T5_BASE = "google/flan-t5-base"

In [5]:
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
import torch
from transformers import AutoTokenizer
import re
import torch
from huggingface_hub import hf_hub_download
from transformers import AutoModel, AutoTokenizer, AutoModelForSeq2SeqLM



C:\Users\madha\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
config = {"persist_directory":None,
          "load_in_8bit":False,
          "embedding" : EMB_SBERT_MPNET_BASE,
          "llm":LLM_FLAN_T5_BASE,
          }

In [7]:
def create_sbert_mpnet():
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Download model and tokenizer config
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
    model_path = hf_hub_download(repo_id="sentence-transformers/all-mpnet-base-v2", filename="pytorch_model.bin")

    # Load model
    model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2", state_dict=torch.load(model_path))
    model.to(device)

    return model, tokenizer



def create_flan_t5_base(load_in_8bit=False):
    model_name = "google/flan-t5-base"

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load the model
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_name,
        device_map="auto" if torch.cuda.is_available() else None,  # Use automatic device placement if CUDA is available
        load_in_8bit=load_in_8bit,  # Load in 8-bit mode if specified
        torch_dtype=torch.float16 if load_in_8bit else None  # Use half-precision in 8-bit mode
    )

    # Wrap in a pipeline
    hf_pipeline = pipeline(
        task="text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=100,
        model_kwargs={"max_length": 512, "temperature": 0.0}
    )

    return hf_pipeline


In [8]:
if config["embedding"] == EMB_SBERT_MPNET_BASE:
    embedding = create_sbert_mpnet()

C:\Users\madha\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
load_in_8bit = config["load_in_8bit"]
if config["llm"] == LLM_FLAN_T5_BASE:
    llm = create_flan_t5_base(load_in_8bit=load_in_8bit)

In [10]:
pdf_path = "cn.pdf"
loader = PDFPlumberLoader(pdf_path)
print(loader)
documents = loader.load()
print(documents)

[Document(page_content='UNIT - I\nNETWORKS\nA network is a set of devices (often referred to as nodes) connected by\ncommunication links. A node can be a computer, printer, or any other device\ncapable of sending and/or receiving data generated by other nodes on the\nnetwork.\n“Computer network’’ to mean a collection of autonomous computers\ninterconnected by a single technology. Two computers are said to be\ninterconnected if they are able to exchange information.\nThe connection need not be via a copper wire; fiber optics, microwaves,\ninfrared, and communication satellites can also be used.\nNetworks come in many sizes, shapes and forms, as we will see later.\nThey are usually connected together to make larger networks, with the\nInternet being the most well-known example of a network of networks.\nThere is considerable confusion in the literature between a computer\nnetwork and a distributed system. The key distinction is that in a distributed\nsystem, a collection of independent c

In [11]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
print(text_splitter)
texts = text_splitter.split_documents(documents)
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=10, encoding_name="cl100k_base")  # This the encoding for text-embedding-ada-002
texts = text_splitter.split_documents(texts)
print(texts)

[Document(page_content='UNIT - I\nNETWORKS\nA network is a set of devices (often referred to as nodes) connected by\ncommunication links. A node can be a computer, printer, or any other device\ncapable of sending and/or receiving data generated by other nodes on the\nnetwork.\n“Computer network’’ to mean a collection of autonomous computers\ninterconnected by a single technology. Two computers are said to be\ninterconnected if they are able to exchange information.\nThe connection need not be via a copper wire; fiber optics, microwaves,\ninfrared, and communication satellites can also be used.\nNetworks come in many sizes, shapes and forms, as we will see later.\nThey are usually connected together to make larger networks, with the\nInternet being the most well-known example of a network of networks.\nThere is considerable confusion in the literature between a computer\nnetwork and a distributed system. The key distinction is that in a distributed\nsystem, a collection of independent c

In [12]:
persist_directory = config["persist_directory"]
vectordb = Chroma.from_documents(documents=texts, embedding_function=embedding, persist_directory=persist_directory)

No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


In [13]:
hf_llm = HuggingFacePipeline(pipeline=llm)
retriever = vectordb.as_retriever(search_kwargs={"k":4})
qa = RetrievalQA.from_chain_type(llm=hf_llm, chain_type="stuff",retriever=retriever)
# , chain_kwargs = {"return_intermediate_steps":True})

In [14]:
if config["llm"] == LLM_FLAN_T5_BASE:
    question_t5_template = """
    context: {context}
    question: {question}
    answer:
    """
    QUESTION_T5_PROMPT = PromptTemplate(
        template=question_t5_template, input_variables=["context", "question"]
    )
    qa.combine_documents_chain.llm_chain.prompt = QUESTION_T5_PROMPT

In [15]:
question = "Explain Bus Topology in detail"
answer = qa({"query":question,})
print(answer)

Token indices sequence length is longer than the specified maximum sequence length for this model (536 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=100) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'query': 'Explain Bus Topology in detail', 'result': 'Bus topology is a network setup in which each computer and network device are connected to a single cable or backbone.'}
